In [ ]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Train/Test 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Optuna 목적 함수 정의
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 250, 350),  # 기존 값 주변으로 범위 설정
        'max_depth': trial.suggest_int('max_depth', 12, 18),         # 기존 값 주변
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.15),
        'subsample': trial.suggest_float('subsample', 0.9, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.9, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),    # 추가적으로 규제 항목 튜닝
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0)
    }
    
    # 모델 초기화
    model = XGBRegressor(random_state=42, **params)
    
    # 학습 및 검증
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_pred)
    
    return mae  # MAE를 최소화

# Optuna 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # 50회 시도

# 최적 파라미터 출력
best_params = study.best_params
print("Best Parameters from Optuna:", best_params)

# 최적 모델로 테스트 데이터 예측
best_model = XGBRegressor(random_state=42, **best_params)
best_model.fit(X_train_full, y_train_full)
y_test_pred = best_model.predict(X_test)

# 결과 저장
result = pd.DataFrame({'id': test_data['id'], 'price': y_test_pred})
result.to_csv('7_pred_optuna.csv', index=False)
print("7_pred_optuna.csv 파일이 생성되었습니다.")
